<a href="https://colab.research.google.com/github/tianxiang84/PINN-elastodynamics/blob/wave_1d_rod/ElasticWaveRod/PINN_Rod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [2]:
import numpy as np
import time
import tensorflow as tf
tf.__version__

'2.4.1'